In [ ]:
# Notebook setup: ensure we run from repo root (so relative paths like configs/ and runs/ work)
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p] + list(p.parents):
        if (cand / "pyproject.toml").exists() and (cand / "src").exists():
            return cand
    # Fallback: if executed from notebooks/, go one level up
    if p.name.lower() == "notebooks" and (p.parent / "src").exists():
        return p.parent
    return p

REPO_ROOT = _find_repo_root()
os.chdir(REPO_ROOT)

src_path = REPO_ROOT / "src"
if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)

# Note:
# This notebook assumes configs/ and runs/ are relative to the repo root.


# FeatureLab — login_attempt


This notebook builds **login_attempt** features for an existing run_id, then inspects the resulting feature table.


In [ ]:
from pathlib import Path
import pandas as pd

from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.features import build_login_features_for_run
from inkswarm_detectlab.io.tables import read_auto

cfg_path = Path("configs/skynet_smoke.yaml")
if not cfg_path.exists():
    cfg_path = Path("configs") / "configs" / "skynet_smoke.yaml"
cfg = load_config(cfg_path)
run_id = cfg.run.run_id or "RUN_XXX_0005"  # <-- change me if needed

# Build (idempotent unless --force)
build_login_features_for_run(cfg, run_id=run_id, force=False)

feat_df = read_auto(Path(cfg.paths.runs_dir) / run_id / "features" / "login_attempt" / "features")
feat_df.head()


In [ ]:
# Basic sanity: label prevalence + a few feature columns
label_cols = [c for c in feat_df.columns if c.startswith("label_")]
feat_df[label_cols].mean().sort_values(ascending=False)


In [ ]:
# Show a couple of features
cols = [c for c in feat_df.columns if "__attempt_cnt" in c][:5] + [c for c in feat_df.columns if c.endswith("failure_rate")][:5]
feat_df[cols].describe().T
